# FHIR Auth Exploration
------------------------------------

Evaluate the authentication and authorization features on a handful of FHIR servers. 

## Background

- See general OAuth 2.0 + OIDC refresher in `docs/explore/auth.md`
- See summary of FHIR auth spec in `docs/explore/fhir_auth.md`

## Summary

Evaluations are listed from least promising to most promising

### Microsoft FHIR Server on Azure

Pros

Cons


### Vonk

Pros

Cons


### Smile CDR (commercial HAPI)

Pros

Cons

### Aidbox

Pros

Cons
- Writing policies with sql is sort of confusing, need more documentation
- Writing policies with json-schema is very confusing and overly verbose
- Writing policies is easiest with matcho engine, but it doesn't work yet
- RBAC implementation is very confusing. A role is not a collection of permissions
  A role is just a tag on a user. The policy looks at that tag among other things
  about the request (method, resource) to determine authorization. 

Questions
- How do you restrict access to nested resources that result from search queries utilizing _include parameter
- How do you restrict access to related resources that can be access via chained searches?
- What is the best way to restrict access to an arbitrary group of resources? Is this what compartments are for?

Comments
- I believe there is a bug with tokens. Aidbox returns opaque token even when client is set to use
  JWT
- I believe there is a bug with how the access policies are evaluated
- Currently Aidbox authorizes access if any([policy eval = True for policy in access_policies])
  I'm not sure if this is good? Maybe we want deny access if any([policy eval = False for policy in   
  access_policies])


In [1]:
import requests
from click.testing import CliRunner
from pprint import pprint, pformat
import pandas

from requests.auth import HTTPBasicAuth

from kf_model_fhir.config import FHIR_VERSION, SERVER_CONFIG, PROJECT_DIR
from kf_model_fhir.loader import load_resources
from kf_model_fhir.utils import read_json

from helpers import *

# Setup Required
--------------------------

Every server being evaluated is publically hosted so you don't need to spin up any docker containers. You just need to clone the `kf-model-fhir` repo and switch to the `search-api-testing` branch

### 1. Get the Code

```shell
# Get code
git clone git@github.com:kids-first/kf-model-fhir.git
cd kf-model-fhir

# Switch to right branch
git checkout auth-feat-testing
```

### 2. Setup Virtual Environment

```shell
# Setup virtual env
python3 -m venv venv
source venv/bin/activate

# Install requirements
pip install -e .
```

Now you're ready to run this notebook

### Important Notes

\* _Your Network Might Block Some FHIR Servers_
```
For us at chop, this means you have to be on `chopguest` to run 
this since `chopnet` blocks the Smile CDR server
```

\* _Disclaimer - Throw Away Code_
```
Code in this branch is throw away code and only meant for search API exploration - don't judge :)
There are probably bugs and things might break if you change certain things
```

### 3. Generate and Load Data

- Data for this notebook is in the `kf-model-fhir/project` folder. 
- Conformance resources like StructureDefinitions and SearchParameters are in `kf-model-fhir/project/profiles` 
- Dummy resources that were generated from the step earlier are located `kf-model-fhir/project/resources`

\* _Only do this if you haven't already loaded data_

In [2]:
# Generate resources
# run_cli_cmd('generate', ['./resources'])
# Load the servers with profiles and resources
# load_all_servers()

# Aidbox - as a Resource + Auth Server
----------------------------------------------------------
- Aidbox is a fully compliant OAuth 2.0 and OpenID connect authentication and authorization server
- Aidbox can also be used as purely an OAuth 2.0 Resource Server, capable of validating OAuth 2 access tokens
  from a 3rd party auth service (e.g. Auth0)
- When used as a SMART on FHIR backend service, Aidbox also supports access control based on SMART on FHIR scopes

In this section:
- Aidbox will serve as the OAuth 2.0 Resource Server
- Aidbox will also serve as the OAuth 2.0 authentication and authorization server

## Auth Discovery API
Before we do anything, we need to know about the various authorization endpoints the Aidbox auth server exposes.
For this we will check the well known URI endpoints. Aidbox as several: https://docs.aidbox.app/auth-betta/well-known-endpoint. The one we care about right now is the OAuth server endpoint: `/.well-known/oauth-authorization-server`

In [4]:
server_settings = SERVER_CONFIG['aidbox-local']
base_url = server_settings['base_url'].rstrip('/fhir')
base_fhir_url = server_settings['base_url']

_, resp_content = client.send_request(
    'get',
    f"{base_url}/.well-known/oauth-authorization-server"
)
token_endpoint = resp_content['response'].get('token_endpoint')

2019-12-02 12:32:53,194 - FhirApiClient - DEBUG - GET http://localhost:8081/.well-known/oauth-authorization-server succeeded. Response:
{'authorization_endpoint': 'http://localhost:8081/auth/authorize',
 'claims_supported': ['sub',
                      'aud',
                      'email',
                      'exp',
                      'iat',
                      'iss',
                      'locale',
                      'family_name',
                      'given_name',
                      'name',
                      'picture'],
 'grant_types_supported': ['authorization_code',
                           'implicit',
                           'password',
                           'client_credentials'],
 'id_token_signing_alg_values_supported': ['RS256'],
 'issuer': 'http://localhost:8081',
 'jwks_uri': 'http://localhost:8081/.well-known/jwks.json',
 'response_types_supported': ['code',
                              'token',
                              'token id_token',
 

## Scenario 1 - Basic Auth Flow
- This notebook will serve as the client app requesting access to FHIR resources
- The app will authenticate using basic auth
- The app will have an access policy which gives it root access to everything

Try to create a new client

In [ ]:
notebook_client_id = 'fhir-auth-notebook-client'
notebook_client_secret = 'mypassword'
endpoint = f"{base_url}/Client/{notebook_client_id}"
payload= {
        'secret': notebook_client_secret,
        'grant_types': ['basic']
}
_, resp_content = client.send_request(
    'put', 
    endpoint,
    json=payload
)

The above step failed because the anonymous user is not authorized to do anything on the auth server
Aidbox comes with a default root client which can do anything. We will use this client for administrative
operations

In [ ]:
# Get the root client creds
root_client_id = server_settings['username']
root_client_pw = server_settings['password']
root_client_auth = HTTPBasicAuth(root_client_id, root_client_pw)

# Resend request with root client creds
endpoint = f"{base_url}/Client/{notebook_client_id}"
_, resp_content = client.send_request(
    'put', 
    endpoint,
    json=payload,
    auth=root_client_auth
)

Try to access patient resources

In [ ]:
_, resp_content = client.send_request(
    'get', f"{base_fhir_url}/Patient"
)

The above failed because there is no access policy for our notebook client. We must create an access policy for the new client (again as the admin/root client)

\* Read more about access control management here: https://docs.aidbox.app/security/access-control

In [ ]:
# Create the access policy for the notebook client
endpoint = f"{base_url}/AccessPolicy/notebook-root-access-policy"
payload = {
    'engine': 'allow',
    'description': 'Allows any operations on any resource',
    'link': [
        {
            'resourceType': 'Client',
            'id': 'fhir-auth-notebook-client'
        }
    ]
}
_, resp_content = client.send_request(
    'put', 
    endpoint,
    json=payload,
    auth=root_client_auth
)

# Try to fetch the patients again
# This time use your notebook client credentials
notebook_client_auth = HTTPBasicAuth(notebook_client_id, notebook_client_secret)
_, resp_content = client.send_request(
    'get', 
    f"{base_fhir_url}/Patient",
    auth=notebook_client_auth 
)
print(f"Resource count: {resp_content['response']['total']}")

## Scenario 2 - Client Credentials OAuth 2.0 Auth Flow
- This notebook will serve as the OAuth 2.0 compliant `confidential client` app requesting access to FHIR resources
- This is for demonstration purposes only. In real life a `confidential client`:
    - Would likely be an internal app/process known to the FHIR resource server
    - Is confidential because it can securely store its secret
- This app will have an access policy which grants it root access to everything
- This app will authenticate using its client id, secret via basic auth
- Upon successful authentication client will receive an access token
- Client will use the access token to make requests to access FHIR resources

### \* Note on Access Token
The access token issued by Aidbox is just a simple JWT with no custom scopes/permissions. All permissions
are defined via the Aidbox AccessPolicy resource

1. Update our notebook `Client` resource with client_credentials grant type and new secret

In [ ]:
notebook_client_secret= 'verysecret'
_, resp_content = client.send_request(
    'put', 
    f"{base_url}/Client/{notebook_client_id}",
    json={
        'secret': notebook_client_secret,
        'grant_types': ['client_credentials'],
        'auth': {
            'client_credentials':{'token_format': 'jwt'}
        }
    },
    auth=root_client_auth
)


2. Already have an access policy for this client, so we don't need to do anything here

3. Authenticate and get access token

In [ ]:
_, resp_content = client.send_request(
    'post', 
    token_endpoint,
    json={
        'grant_type': 'client_credentials',
        'client_id': notebook_client_id,
        'client_secret': notebook_client_secret
    }
)

jwt = resp_content['response']['access_token']

4. Use the access token to get patients

In [ ]:
_, resp_content = client.send_request(
    'get', f"{base_url}/Patient",
    headers={
        'Authorization': f'Bearer {jwt}'
    } 
)
print(f"Resource count: {resp_content['response']['total']}")

## Aidbox Access Control

- Aidbox seems to mostly support Attribute Based Access Control (ABAC) via its access policies
- Aidbox has several ways to write access policies/rules
    - Use `engine: allow` which means allow everything
    - Using `engine: sql`
    - Using `engine: json-schema`
    - Using `engine: complex` (allows composition of multiple policies)
    - Using the new Aidbox policy engine called `matcho` which is based on custom DSL
- An access policy can be appied to one or more `User`, `Client`, or `Operation` Aidbox resources
- An incoming HTTP request is parsed into an Aidbox request object
  See https://docs.aidbox.app/security/access-control for details
- Whichever policy "engine" you choose you have access to all of the attributes
  in the _synthesized_ request object
- The request object aggregates things like: user and/or client credentials, user roles, request body, request   
  query string params, etc into one object 
- Aidbox will find all of the access policies for the user and/or client associated with the request object
- Aidbox will evaluate each access policy against the request object - as soon as a policy evaluates to `TRUE`, 
  the request is authorized

### Scenario 1: Simple Access Control

#### Read Only Policy for Clients

- Delete the root policy for clients. 
- Create an access policy that allows read access to any resource by any client

In [ ]:
# Create new read all resources policy for notebook client
endpoint = f"{base_url}/AccessPolicy/read-all-resources-policy"
payload = {
    'engine': 'json-schema',
    'schema': {
        'required': ['uri', 'client', 'request-method'],
        'properties': {
            'uri': {
                'type': 'string',
                'pattern': '^/fhir/.*'
            },
            'request-method': {
                'const': 'get'
            }
        }
    },
    'description': 'Allows GET operations on any resource by any client'
}
_, resp_content = client.send_request(
    'put', 
    endpoint,
    json=payload,
    auth=root_client_auth
)

Make another client to test this out

In [ ]:
_, resp_content = client.send_request(
    'put', 
    f"{base_url}/Client/test-client-id",
    json={
        'secret': 'secret',
        'grant_types': ['client_credentials'],
        'auth': {
            'client_credentials':{'token_format': 'jwt'}
        }
    },
    auth=root_client_auth
)

Test each client's ability to GET and POST patients. POSTs should fail for both clients bc there is no policy for it

In [ ]:
for c in [('test-client-id', 'secret'), (notebook_client_id, notebook_client_secret)]:
    # Authenticate client and get access token
    _, resp_content = client.send_request(
        'post', 
        token_endpoint,
        json={
            'grant_type': 'client_credentials',
            'client_id': c[0],
            'client_secret': c[1]
        }
    )
    jwt = resp_content['response']['access_token']
    
    # Get patients
    success, resp_content = client.send_request(
        'get', f"{base_fhir_url}/Patient",
        headers={
            'Authorization': f'Bearer {jwt}'
        } 
    )

    print(f"\n✅ GET {resp_content['request_url']} Resource count: {resp_content['response']['total']}")
          
    # POST patient - should fail
    success, resp_content = client.send_request(
        'put', f"{base_fhir_url}/Patient/test-patient",
        headers={
            'Authorization': f'Bearer {jwt}'
        },
        json={
            'resourceType': 'Patient',
            'name': [
                {
                    'given': ['test-patient']
                }
            ] 
        }
    )
    if not success:
          print(f"\n❌ POST Failed - {resp_content}")

### Scenario 2: Role Based User Access Control

#### Developer Role 
- Can create, read, update any FHIR resource
- Can delete resources they created (not sure how to do this)

#### Admin Role
- Full CRUD access to all FHIR resources

#### Explorer Role
- Can read FHIR resources for research studies they have access to

In [ ]:
# Get studies
_, resp = client.send_request(
    'get',
    f'{base_fhir_url}/ResearchStudy',
    auth=root_client_auth
)
studies = [s['resource']['id'] for s in resp['response']['entry']]

# Create a user for each study 
for i, study_id in enumerate(studies):
    user_id = f'user{i}'
    _, resp = client.send_request(
        'put',
        f"{base_url.rstrip('/fhir')}/User/{user_id}",
        auth=root_client_auth,
        json={
            'data': {
                'name': user_id,
                'study_id': study_id,
                'roles': ['Explorer']
            },
            'email': f"{user_id}@chop.edu",
            'password': user_id,
            'id': user_id,
            'resourceType': 'User'
        }
    )

# Delete the previous policy
_, resp_content = client.send_request(
    'delete', 
    f"{base_url}/AccessPolicy/read-all-resources-policy",
    auth=root_client_auth
)

# Create an access policy and attach to Explorer role
_id = 'read-only-research-subject-policy'
policy = f"""
    sql:
      query: |
        SELECT
          {{user}} IS NOT NULL
          AND {{jwt}} IS NOT NULL
          AND {{user.data.roles}} IS 'Explorer'
          AND {{user.data.study_id}} IS NOT NULL
          AND {{uri}} LIKE '/fhir/ResearchSubject.*'
          AND resource->'study' @>
        jsonb_build_array(jsonb_build_object('resourceType',
            'ResearchSubject', 'id', {{user.data.study_id}}::text))
          FROM researchsubject;
    engine: sql
    id: {_id}
    resourceType: AccessPolicy
"""
_, resp = client.send_request(
    'put',
    f'{base_url}/AccessPolicy/{_id}',
    auth=root_client_auth,
    headers={
        'Content-Type': 'text/yaml'
    },
    data=policy
)

# Add password grant to notebook client
_, resp = client.send_request(
    'patch',
    f'{base_url}/Client/{notebook_client_id}',
    json={
        'grant_types': ['client_credentials', 'password']
    },
    auth=root_client_auth
)
# Get token first
_, resp = client.send_request(
    'post',
    token_endpoint,
    json={
        'client_id': notebook_client_id,
        'client_secret': notebook_client_secret,
        'username': 'user1',
        'password': 'user1',
        'grant_type': 'password'
    }
)
access_token = resp['response']['access_token'] 

In [ ]:
# Get research subjects
_, resp = client.send_request(
    'get',
    f"{base_url}/ResearchSubject",
    headers={
        'Authorization': f"Bearer {access_token}"
    },
    params={'__debug': 'policy'}
)

# Aidbox - with External Auth Service (Auth0)
--------------------------------------------------------------------